In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import scipy.signal as sg
import scipy as sp
from mpl_toolkits import mplot3d
import ellipses as el
from matplotlib.patches import Ellipse
import sys
%matplotlib inline
# helper functions
# ellipse fitting function
def efit(data, maxpR, right=True):
    lsqe = el.LSqEllipse()
    lsqe.fit(data)
    center, width, height, phi = lsqe.parameters()
    vector = [(center[0]-maxpR[1]), (center[1]-maxpR[0])]
    vecmag = np.sqrt(vector[0]**2+vector[1]**1)
    return center, vector
# remove noise
def debubble(cpos, cneg, rpos, rneg, win_):
#     in# put: xcor of edge points
#     ncol: number of column
#     posfirst: posedge happens before negedge
    counts = np.bincount(cpos)
    pmode = np.argmax(counts)
    counts2 = np.bincount(cneg)
    nmode = np.argmax(counts2)
    ta1 = np.zeros(len(cpos)).astype('bool')
    ta2 = np.zeros(len(cneg)).astype('bool')
    print('truncation: ', pmode, '+-', win_)
    print('truncation: ', nmode, '+-', win_)
    for i in range(len(cpos)):
        ta1[i] = (cpos<(pmode+win_))[i] and (cpos>(pmode-win_))[i]
    for i in range(len(cneg)):
        ta2[i] = (cneg<(nmode+win_))[i] and (cneg>(nmode-win_))[i]
    return cpos[ta1], cneg[ta2], rpos[ta1], rneg[ta2]
# imgEye : list of 2d array, image
# frame: 2d array
# facecor: (2,)array, upper right cor of face frame
# eye: t*4 array, list of eyes' cor, under face frame
# parameters:
zoomout = 1 # 
ratio = .32
narrower = 0 
pns = 50
nns = 50
win = 10 # bilateral window size in pixel
print ('modules imported')

modules imported


In [2]:
#   faces eyes detection
def getEyes(frame):
    face_cascade = cv.CascadeClassifier('C:\\Users\\wangy\\Anaconda2\\Lib\\site-packages\\cv2\\data\\haarcascade_frontalface_default.xml')
    eye_cascade = cv.CascadeClassifier('C:\\Users\\wangy\\Anaconda2\\Lib\\site-packages\\cv2\\data\\haarcascade_eye.xml')
    if (eye_cascade.empty()==True or face_cascade.empty()==True):
        print ('Fail to load xml data')
        return []
    imgEye = []
    cnt = 0
    img = frame
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    if(len(faces) != 1):
        print("Face dectection failure")
        return []
    for (x,y,w,h) in faces:
        cv.rectangle(img,(x,y),(x+w,y+h),(255,0,0),5)
        roi_gray = gray[y:y+h, x:x+w]
#         cv.imwrite('pic'+str(counter)+'.png', frame )
        roi_color = img[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)
        for (ex,ey,ew,eh) in eyes:
            imgEye.append(cv.cvtColor(roi_color[ey+narrower:ey+(int)(ew), ex:ex+(int)(eh*zoomout)], cv.COLOR_BGR2GRAY))
            cv.putText(roi_color, str(cnt), (ex-20, ey-20), cv.FONT_HERSHEY_SIMPLEX, 3, (0,0,255),thickness=5)
            cnt += 1
    return imgEye, frame, eyes, faces[0,0:2]
print('function imported !')

function imported !


In [3]:
def showGaze(p1, p2, counter):
    imgEye[0] = cv.cvtColor(frame[p1[1]:p2[1], p1[0]:p2[0]], cv.COLOR_BGR2GRAY)
#     cv.imwrite('eye'+str(counter)+'.png', imgEye[0])

    cfR = np.array([[-1,-1,-1,1,1,1],[-1,-1,-1,-1,1,1],[-1,-1,-1,-1,-1,1],[1,1,1,1,1,1]])
    print ((imgEye[0][:,:(int)(imgEye[0].shape[1]*ratio)]).shape)
    foR = sg.convolve2d(imgEye[0][:,:(int)(imgEye[0].shape[1]*ratio)], cfR, mode='valid')
    maxpR = np.array([(int) (np.argmax(foR)/foR.shape[1]), np.argmax(foR)%foR.shape[1]])
    gradx = cv.Sobel(imgEye[0],cv.CV_64F,1,0,ksize=5)
    gradx = gradx[:,(int)(imgEye[0].shape[1]*ratio):]
    row, col = gradx.shape[0], gradx.shape[1]
    ranks = np.argsort(gradx.flatten())
    cargp = (ranks[-pns:] % col).astype('int')
    rargp = (ranks[-pns:] / col).astype('int')
    cargp += (int) ((imgEye[0].shape[1])*ratio)
    cargn = (ranks[0:nns] % col).astype('int')
    rargn = (ranks[0:nns] / col).astype('int')
    cargn += (int)((imgEye[0].shape[1])*ratio)
    cargp_, cargn_, rargp_, rargn_ = debubble(cargp, cargn, rargp, rargn, win)
    xcor = np.append(cargp_, cargn_)
    ycor = np.append(rargp_, rargn_)
    for X in xcor: 
        for Y in ycor:
            pointx = (int) (X) + p1[0]
            pointy = (int) (Y) + p1[1]
#             cv.circle(frame, (pointx, pointy), 1, (255,0,0)) 
    center, _ = efit ([xcor, ycor], maxpR)
    arre = (p1[0] + (int) (maxpR[1]), p1[1] + (int) (maxpR[0]))
    arrs = (p1[0] + (int) (center[1]), p1[1] + (int) (center[0]))
    cv.arrowedLine(frame, arrs, arre, (0,0,255), 3)
    cv.circle(frame, (arrs), 3, (0,255,0)) # green RGB
    cv.circle(frame, (arre), 3, (255,0,127)) # purple 
    print (maxpR[1], maxpR[0], center[0], center[1])
#     cv.imwrite('face'+str(counter)+'.png', frame )
    counter += 1
    vector = [maxpR[1]-center[1], maxpR[0]-center[0]]
    return frame, vector, counter
print('function imported !')

function imported !


In [4]:
video = cv.VideoCapture(0)
counter = 0
if not video.isOpened():
    print('Could not open video!')
    sys.exit()
tracker = None
roi = (0, 0, 0, 0)
predx = [0,0,0,0]
predy = [0,0,0,0]
outcx = [0,0,0,0]
outcy = [0,0,0,0]
c0x = 0
c1x = 0
c0y = 0
c1y = 0
calibrate_flag = 0
tracking_flag = -1
while True:
    key = cv.waitKey(1) & 0xff
    read_success, frame = video.read()
#     frame = np.fliplr(frame)
    if not read_success:
        print('Cannot read video file!')
        sys.exit()
    if key == 13:
        if tracking_flag == -1:
        # calibrate in order upperleft, upperright, bottomleft, bottomright
            outcx = [0,              0, (int) (frame.shape[1] / 2), frame.shape[1]]
            outcy = [0, frame.shape[0], (int) (frame.shape[0] / 2), frame.shape[0]]

            imgEye, frame, eye, facecor = getEyes(frame)
            cv.imshow("ECE420 Lab7", frame)
            key = cv.waitKey(0) & 0xff
            if (key == 13):
                continue
            if (key-48 >= len(imgEye)): 
                print("Cannot find eye!")
                sys.exit()
            roi = eye[key-48]
            roi = tuple(roi + np.append(facecor, [0,0]))
            tracking_flag = 0
        if tracking_flag == 1:
            roi = (0, 0, 0, 0)
            tracker.clear()
            reye = None
            tracking_flag = -1
    elif key == 27:
        break
    start = cv.getTickCount()
    if tracking_flag == -1:
        cv.putText(frame, "Press ENTER to select ROI!", (10, 30), cv.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
    elif tracking_flag == 0:
        # Initialize KCF Tracker and Start Tracking
        # 1. Create a KCF Tracker
        # 2. Initialize KCF Tracker with grayscale image and ROI
        # 3. Modify tracking flag to start tracking
        # Your code starts here
        tracker = cv.TrackerMedianFlow_create()
        tracker.init(frame, roi)
        tracking_flag = 1
    else:
        # Update tracking result is succeed
        # If failed, print text "Tracking failure occurred!" at top left corner of the frame
        # Calculate and display "FPS@fps_value" at top right corner of the frame
        # Your code starts here
        (success, roi) = tracker.update(frame)
        if success:
            p1 = (int(roi[0]), int(roi[1]))
            p2 = (int(roi[0] + roi[2]), int(roi[1] + roi[3]))
            frame, vector, counter = showGaze(p1,p2, counter)
            if (calibrate_flag <= 3): # calibrate
                key = cv.waitKey(0) & 0xff
                if (key == 13):
                    cv.imshow("ECE420 Lab7", frame)
                    key = cv.waitKey(0) & 0xff
                    if (key == 13): 
                        print('calibrating point', calibrate_flag)
                        predx[calibrate_flag] = vector[0]
                        predy[calibrate_flag] = vector[1]
                        calibrate_flag += 1
            elif (calibrate_flag == 4):
                # linear regression outcome = c1 * income + c0 
                vandx = np.vander(predx, 2)[:, ::-1]
                vandy = np.vander(predy, 2)[:, ::-1]
                lrsolx,_,_,_ = np.linalg.lstsq(vandx, outcx)
                lrsoly,_,_,_ = np.linalg.lstsq(vandy, outcy)
                c0x = lrsolx[0]
                c1x = lrsolx[1]
                c0y = lrsoly[0]
                c1y = lrsoly[1]
                calibrate_flag += 1
                plt.figure()
#                 plt.subplot(121)
                plt.plot(predx, predy,'ro')
#                 plt.plot(c0x + predx * c1x)
                plt.title('linregres for x cors')
                plt.xlabel('predictor x')
                plt.ylabel('predictor y')
#                 plt.subplot(122)
#                 plt.plot(predy, outcy,'bo')
# #                 plt.plot(c0y + predy * c1y)
#                 plt.title('linregres for y cors')
                plt.savefig('linregres_xy.png', dpi=256)
            else:
                cv.rectangle(frame, p1, p2, (255, 0, 0), 2, 1)
    #             cv.imwrite('eye'+str(counter)+'.png', frame[p1[1]:p2[1], p1[0]:p2[0]])
                cursorx = vector[0] * c1x + c0x
                cursory = vector[1] * c1y + c0y
                print('cursor at',cursorx,',',cursory)
                if(cursorx>frame.shape[1] or cursorx<0 or cursory>frame.shape[0] or cursory<0):
                    cv.putText(frame, "Cursor Out Of Frame", (10, 30), cv.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
                cv.circle(frame, ((int) (cursorx), (int) (cursory)), 8, (255,0,0), thickness=4) # order of cor not sure
        else:
            cv.putText(frame,"Tracking failure occured!",(10,20),cv.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
    fps = cv.getTickFrequency() / (cv.getTickCount() - start);
    cv.putText(frame, "FPS : " + str(int(fps)), (100, 20), cv.FONT_HERSHEY_SIMPLEX, 0.75, (50, 170, 50), 2)
    cv.imshow("ECE420 Lab7", frame)